In [27]:
import cv2 as cv
import numpy as np
import keyboard
import time
import math


import Controlling_thymio
import global_planning
import vision

In [47]:
# Vision example on an image

# load the image
img = cv.imread("testing_vision.jpg")
img = cv.resize(img, (640,480), interpolation=cv.INTER_CUBIC) 

# do the actual vision
original_coordinates, new_coordinates = vision.get_fop_coordinates(img)
fop = vision.get_fop(img, original_coordinates, new_coordinates)

start_x, start_y, alpha, width = vision.get_robot_position(fop)
obstacles = vision.get_obstacles(fop,width)
objective_x, objective_y = vision.get_objective(fop)

# draws everything and displays it
output = fop.copy()
cv.circle(output, (objective_x, objective_y), radius=10, color=(255, 0, 0), thickness=-1)
cv.circle(output, (start_x, start_y), radius=10, color=(0, 0, 255), thickness=-1)
for obstacle in obstacles:
    for vertice in obstacle:
        cv.circle(output, (vertice[0],vertice[1]), radius=10, color=(0, 255, 0), thickness=-1)

print("Angle:" + str(alpha))

converted_data = [[tuple(arr) for arr in sublist] for sublist in obstacles]
print(converted_data)

converted_data = global_planning.obstacle_dictionnary(converted_data)
robot_instance = global_planning.Robot()
goal = (objective_x,objective_y)

robot_instance.update_coordinates(start_x, start_y, alpha,width)
obstacles_named = global_planning.naming_points(converted_data,robot_instance,goal)
adg_list = global_planning.creating_adjacency_dictionnary(converted_data,robot_instance,goal)
path = global_planning.finding_path(adg_list,obstacles_named)
print(path)
for i in range(len(path[:-1])):
    cv.line(output,(obstacles_named[path[i]][0],obstacles_named[path[i]][1]),(obstacles_named[path[i+1]][0],obstacles_named[path[i+1]][1]),(255,0,0),5)
    

cv.imshow("image", output)

# press any key to close all windows
cv.waitKey(0) 
cv.destroyAllWindows()

Angle:[2.65711472]
[[(np.int32(440), np.int32(313)), (np.int32(344), np.int32(266)), (np.int32(372), np.int32(141))], [(np.int32(295), np.int32(140)), (np.int32(193), np.int32(281)), (np.int32(18), np.int32(128))]]
['R', 'P2', 'P4', 'G']


In [3]:
print("Dimensions of the rectangular frame:")
print(new_coordinates[3][0], new_coordinates[3][1])

print("Starting coordinates and angle:") # angle of the robot relative to the x axis, counterclockwise, expressed in radian in range (-pi, pi]
print("(x ,y, alpha) = " + str(start_x) + ", "+ str(start_y) + ", " + str(alpha))

print("Robot width:")
print(width)

print("Vertices of the obstacles:")
print(obstacles)

print("Goal coordinates:")
print(objective_x, objective_y)


Dimensions of the rectangular frame:
580.0 369.0
Starting coordinates and angle:
(x ,y, alpha) = 533, 48, [2.65711472]
Robot width:
62
Vertices of the obstacles:
[[array([502, 134], dtype=int32), array([433, 312], dtype=int32), array([344, 269], dtype=int32), array([372, 140], dtype=int32)], [array([299, 138], dtype=int32), array([193, 281], dtype=int32), array([ 21, 141], dtype=int32), array([18, 92], dtype=int32)]]
Goal coordinates:
42 324


In [3]:
# Global constants here

# width of the video feed
IMAGE_WIDTH = 640
# height of the video feed
IMAGE_HEIGHT = 480
# fps of the video feed
CAMERA_FPS = 30
# keyboard input to start
KEYBOARD_INPUT = "enter"
# number of frame thrown away to allow the camera to focus in the meantime
CAMERA_REFRESH_TIME = 30

In [ ]:
# 0 is laptop webcam, 1 is USB camera (!COMPUTER DEPENDENT!)
cap = cv.VideoCapture(0)

init = False # boolean set to True on keyboard input to start initialisation
is_init = False # boolean set to True once initialisation has succeeded
black_frame_remaining = 0 # numbers of frames to throw away
being_kidnapped = False # boolean set to True while the robot is being kidnapped

# variable initialized at False irrelevant of wether it actually is
camera_hidden = False

# create a robot instance
robot_instance = global_planning.Robot()

# if unable to connect to the camera
if not (cap.isOpened()):
    print("Could not open video device")
else:
    while(True):
        # get the input frame by frame (shape (480,640,3))
        ret, frame = cap.read() 
        cap.set(cv.CAP_PROP_FPS, CAMERA_FPS) 
        frame = cv.resize(frame, (IMAGE_WIDTH,IMAGE_HEIGHT), interpolation=cv.INTER_CUBIC) 

        # wait until keyboard input to initialize vision
        if keyboard.is_pressed(KEYBOARD_INPUT):
            init = True

        """ # reinitialize the robot after a kidnapping
        if being_kidnapped: 
            if not Controlling_thymio.kidnapping(node):
                
                # to give time to the kidnapper to get out of the frame
                time.sleep(1)

                # reinitialize the starting position of the robot
                fop = vision.get_fop(frame, original_coordinates, new_coordinates)
                start_x, start_y, alpha, width = vision.get_robot_position(fop)

                # reinitialize the global planning
                converted_obstacles = [[tuple(arr) for arr in sublist] for sublist in obstacles]
                converted_obstacles = global_planning.obstacle_dictionnary(converted_obstacles)
                goal = (objective_x, objective_y)
                robot_instance.update_coordinates(start_x, start_y, alpha, width)
                obstacles_named = global_planning.naming_points(converted_data, robot_instance, goal)
                adg_list = global_planning.creating_adjacency_dictionnary(converted_data, robot_instance, goal)
                path = global_planning.finding_path(adg_list, obstacles_named)

                is_init = True
                being_kidnapped = False """
        
        # initialize 
        if init == True:

            try:
                # vision: start, obstacles and goal coordinates
                original_coordinates, new_coordinates = vision.get_fop_coordinates(frame)
                fop = vision.get_fop(frame, original_coordinates, new_coordinates)
                start_x, start_y, alpha, width = vision.get_robot_position(fop)
                obstacles = vision.get_obstacles(fop, width)
                objective_x, objective_y = vision.get_objective(fop)

                # global planning
                converted_obstacles = [[tuple(arr) for arr in sublist] for sublist in obstacles]
                converted_obstacles = global_planning.obstacle_dictionnary(converted_obstacles)
                goal = (objective_x,objective_y)
                robot_instance.update_coordinates(start_x, start_y, alpha,width)
                obstacles_named = global_planning.naming_points(converted_obstacles,robot_instance,goal)
                adg_list = global_planning.creating_adjacency_dictionnary(converted_obstacles,robot_instance,goal)
                path = global_planning.finding_path(adg_list,obstacles_named)
                print(path)
                is_init = True
            except Exception as e:
                print(e)
                print("Initialisation failed")

            init = False

        if is_init:  
        ################################## CALL FUNCTIONS HERE ################################## 
            
            # get out of the loop if the robot is being kidnapped
            """ if Controlling_thymio.kidnapping(node):
                being_kidnapped = True
                is_init = False
                continue """

            fop = vision.get_fop(frame, original_coordinates, new_coordinates)
            try:
                center_x, center_y, alpha, width = vision.get_robot_position(fop)
            except:
                None
            
            # camera was covered, now uncovered
            if center_x != 0 and center_y != 0 and camera_hidden:
                camera_hidden = False
                black_frame_remaining = CAMERA_REFRESH_TIME # to give it time to gain focus

            # camera was uncovered, now covered -> navigation using Kalman filter
            elif center_x == 0 and center_y == 0:
                camera_hidden = True

                # DO KALMAN NAVIGATION HERE

            # camera uncovered and usable -> navigation using vision
            elif black_frame_remaining <= 0:
                fop = cv.circle(fop, (center_x,center_y), radius=10, color=(0, 255, 0), thickness=-1)

                # DO VISION NAVIGATION HERE
                
            else:
                # set the first frames after uncovering to black to allow time to focus
                fop = cv.cvtColor(fop, cv.COLOR_BGR2GRAY)
                _, fop = cv.threshold(fop,255,255,cv.THRESH_BINARY)
                black_frame_remaining -= 1


            # draw everything
            if not camera_hidden and black_frame_remaining <= 0:
                cv.circle(fop, (objective_x, objective_y), radius=10, color=(255, 0, 0), thickness=-1)
                cv.circle(fop, (start_x, start_y), radius=10, color=(0, 0, 255), thickness=-1)
                for obstacle in obstacles:
                    for vertice in obstacle:
                        cv.circle(fop, (vertice[0],vertice[1]), radius=10, color=(0, 255, 0), thickness=-1)
                for i in range(len(path[:-1])):
                    cv.line(fop,(obstacles_named[path[i]][0],obstacles_named[path[i]][1]),(obstacles_named[path[i+1]][0],obstacles_named[path[i+1]][1]),(255,0,0),5)
    

        #########################################################################################
        else:
            # to output the raw feed
            fop = frame

        cv.imshow('Raw',frame)
        cv.imshow('Processed',fop)

        #await client.sleep(0.001)
        
        # Close all windows and get out of the loop if ESC is pressed
        if cv.waitKey(1) == 27:
            cv.destroyAllWindows()
            break

['R', 'P6', 'G']
['R', 'P8', 'G']
division by zero
Initialisation failed
['R', 'P9', 'G']
['R', 'P10', 'G']
['R', 'P8', 'G']
['R', 'P9', 'G']
['R', 'P9', 'G']
['R', 'P7', 'G']
['R', 'P8', 'G']
['R', 'P7', 'G']
['R', 'P10', 'G']
['R', 'P10', 'G']
['R', 'P12', 'G']
['R', 'P8', 'G']
['R', 'P10', 'G']
['R', 'P6', 'G']
['R', 'P11', 'G']
['R', 'P9', 'G']
['R', 'P6', 'G']
['R', 'P10', 'G']
['R', 'P8', 'G']
['R', 'P7', 'G']
['R', 'P10', 'G']
['R', 'P7', 'G']
['R', 'P9', 'G']
['R', 'P10', 'G']
['R', 'P9', 'G']
['R', 'P8', 'G']


In [14]:
from tdmclient import ClientAsync, aw
client = ClientAsync()
import Controlling_thymio

node = await client.wait_for_node()

await node.lock()
aw(node.lock())

await node.wait_for_variables({"acc"})

NodeLockError: Node lock error (current status: connected)

In [ ]:
#PI controller
KP_LINEAR = 5
KI_LINEAR = 0.5
KP_ANGULAR = 15
KI_ANGULAR = 1.5
PATH_DELTA = 5

error_linear = []
error_angle = []

def get_linear_error(start_point, end_point, robot_center):
    """
    Get the euclidean distance between the robot and its optimal trajectory
    Input:
        - start_point: last point in the path the robot went through
        - end_point: next point in the path
        - robot_center: coordinates of the center of the robot
    Output:
        - linear_error: euclidean distance between the robot and its optimal trajectory
    """
    linear_error = np.linalg.norm(np.cross(end_point-start_point, start_point-robot_center))/np.linalg.norm(end_point-start_point)

    # if positive the point is on the left side, if negative the point is on the right side
    side = (robot_center[0]-start_point[0])*(end_point[1]-start_point[1]) - (robot_center[1]-start_point[1])*(end_point[0]-start_point[0])

    return linear_error * np.sign(side)


def get_angular_error(start_point, end_point, robot_angle):
    """
    Get the angle between the robot direction and its optimal trajectory
    Input:
        - start_point: last point in the path the robot went through
        - end_point: next point in the path
        - robot_angle: angle of the robot
    Output:
        - angular_error: angle between the robot direction and its optimal trajectory, in radian between ]-pi;pi]
    """
    # compute the trajectory's angle
    alpha = np.angle([end_point[0] - start_point[0] - (end_point[1] - start_point[1])*1j]) # minus sign for complex part because y-axis going downward
    angular_error = alpha - robot_angle

    if angular_error > math.pi:
        angular_error -= 2*math.pi

    if angular_error < math.pi:
        angular_error += 2*math.pi 

    return angular_error


def PI_controller(error, kp, ki):
    """
    Use a PI controller to compute the system input to correct the robot's trajectory
    Input:
        - error: difference between optimal trajectory and actual one 
        - kp: proportional gain of the controler
        - ki: integral gain of the controler
    Output:
        - system_input: input to give to the robot
    """
    system_input = kp * error[-1] + ki * np.sum(error) / len(error)

    return system_input


def reached_linear_target(end_point, robot_center, delta):
    """
    Check if the robot reached its next target point
    Input:
        - end_point: next target point of the robot
        - robot_center: coordinates of the center of the robot
    Output:
        - True if the robot reach end_point, else False
    """
    remaining_path = np.sqrt((end_point[0]-robot_center[0])**2 + (end_point[1]-robot_center[1])**2)
    if remaining_path < delta:
        return True

    return False


start_point = np.array([obstacles_named[path[0]][0],obstacles_named[path[0]][1]])
end_point = np.array([obstacles_named[path[1]][0],obstacles_named[path[1]][1]])
robot_center = np.array([start_x, start_y]) #(center_x, center_y)
try:
    next_point = np.array([obstacles_named[path[2]][0],obstacles_named[path[2]][1]])
except:
    next_point = [0,0]

if reached_linear_target(end_point, robot_center, PATH_DELTA):
    
    # if the last point has been reached
    if next_point[0] == [0] and next_point[1] == [0]:
        print("Target reached")
    else:
        
        # check if target angle has been reached
        if get_angular_error(end_point, next_point, alpha) < 0.15:
            #path = path[1:]
            print("Angle reached")
        else:
            error_angle = get_angular_error(end_point, next_point, alpha)
            left_wheel_speed = -1 * np.sign(error_angle) * 100
            right_wheel_speed = np.sign(error_angle) * 100

            print(left_wheel_speed, right_wheel_speed)

else:

    error_linear.append(Controlling_thymio.get_linear_error(start_point, end_point,robot_center))
    error_angle.append(Controlling_thymio.get_angular_error(start_point, end_point,alpha))

    left_wheel_speed = 150 - Controlling_thymio.PI_controller(error_angle, KP_ANGULAR, KI_ANGULAR) + Controlling_thymio.PI_controller(error_linear, KP_LINEAR, KI_LINEAR)
    right_wheel_speed = 150 + Controlling_thymio.PI_controller(error_angle, KP_ANGULAR, KI_ANGULAR) - Controlling_thymio.PI_controller(error_linear, KP_LINEAR, KI_LINEAR)

    print(left_wheel_speed, right_wheel_speed)

[133.36315248] [166.63684752]


In [ ]:
start_point = np.array([obstacles_named[path[0]][0],obstacles_named[path[0]][1]])
end_point = np.array([obstacles_named[path[1]][0],obstacles_named[path[1]][1]])

try:
    next_point = np.array([obstacles_named[path[2]][0],obstacles_named[path[2]][1]])
except:
    next_point = None
robot_center = np.array([start_x, start_y]) #(center_x, center_y)

# check if the next point as been reached
if Controlling_thymio.reached_linear_target(start_point, end_point, PATH_DELTA):
    # if the last point has been reached
    if next_point == None:
        print("Target reached")
    else:
        
        if Controlling_thymio.reached_angular_target(end_point, next_point, alpha, ANGULAR_DELTA):
            path = path[1:]
        else:
            error_angle = Controlling_thymio.get_angular_error(end_point, next_point, alpha)
            left_wheel_speed = -1 * np.sign(error_angle) * TURNING_SPEED
            right_wheel_speed = np.sign(error_angle) * TURNING_SPEED
else:

    error_linear.append(Controlling_thymio.get_linear_error(start_point, end_point,robot_center))
    error_angle.append(Controlling_thymio.get_angular_error(start_point, end_point,2.65))

    left_wheel_speed = STRAIGHT_SPEED - Controlling_thymio.PI_controller(error_angle, KP_ANGULAR, KI_ANGULAR) + Controlling_thymio.PI_controller(error_linear, KP_LINEAR, KI_LINEAR)
    right_wheel_speed = STRAIGHT_SPEED + Controlling_thymio.PI_controller(error_angle, KP_ANGULAR, KI_ANGULAR) - Controlling_thymio.PI_controller(error_linear, KP_LINEAR, KI_LINEAR)


In [ ]:
# 0 is laptop webcam, 1 is USB camera (!COMPUTER DEPENDENT!)
cap = cv.VideoCapture(0)

init = False # boolean set to True on keyboard input to start initialisation
is_init = False # boolean set to True once initialisation has succeeded
black_frame_remaining = 0 # numbers of frames to throw away
being_kidnapped = False # boolean set to True while the robot is being kidnapped

# variable initialized at False irrelevant of wether it actually is
camera_hidden = False

# create a robot instance
robot_instance = global_planning.Robot()

# if unable to connect to the camera
if not (cap.isOpened()):
    print("Could not open video device")
else:
    while(True):
        # get the input frame by frame (shape (480,640,3))
        ret, frame = cap.read() 
        cap.set(cv.CAP_PROP_FPS, CAMERA_FPS) 
        frame = cv.resize(frame, (IMAGE_WIDTH,IMAGE_HEIGHT), interpolation=cv.INTER_CUBIC) 

        # wait until keyboard input to initialize vision
        if keyboard.is_pressed(KEYBOARD_INPUT):
            init = True

        # initialize 
        if init == True:

            try:
                # vision: start, obstacles and goal coordinates
                original_coordinates, new_coordinates = vision.get_fop_coordinates(frame)
                fop = vision.get_fop(frame, original_coordinates, new_coordinates)
                start_x, start_y, alpha, width = vision.get_robot_position(fop)
                obstacles = vision.get_obstacles(fop, width)
                objective_x, objective_y = vision.get_objective(fop)

                # global planning
                converted_obstacles = [[tuple(arr) for arr in sublist] for sublist in obstacles]
                converted_obstacles = global_planning.obstacle_dictionnary(converted_obstacles)
                goal = (objective_x,objective_y)
                robot_instance.update_coordinates(start_x, start_y, alpha,width)
                obstacles_named = global_planning.naming_points(converted_obstacles,robot_instance,goal)
                adg_list = global_planning.creating_adjacency_dictionnary(converted_obstacles,robot_instance,goal)
                path = global_planning.finding_path(adg_list,obstacles_named)
            
                is_init = True
            except Exception as e:
                
                print("Initialisation failed: " + str(e))

            init = False

        if is_init:  
        ################################## CALL FUNCTIONS HERE ################################## 


            fop = vision.get_fop(frame, original_coordinates, new_coordinates)
            center_x, center_y, alpha, width = vision.get_robot_position(fop)
            robot_instance.update_coordinates(center_x,center_y,alpha,width)
            vl, vr = Controlling_thymio.compute_wheel_speeds(robot_instance,(obstacles_named[path[1]][0],obstacles_named[path[1]][1]),robot_instance.robot_width,0.05,0.05,0.05,0.02)
            Controlling_thymio.controlling_wheels_speed(int(vl),int(vr),aw,node)
            
            fop = cv.circle(fop, (center_x,center_y), radius=10, color=(0, 255, 0), thickness=-1)
            cv.circle(fop, (objective_x, objective_y), radius=10, color=(255, 0, 0), thickness=-1)
            cv.circle(fop, (start_x, start_y), radius=10, color=(0, 0, 255), thickness=-1)
            for obstacle in obstacles:
                for vertice in obstacle:
                    cv.circle(fop, (vertice[0],vertice[1]), radius=10, color=(0, 255, 0), thickness=-1)
            for i in range(len(path[:-1])):
                cv.line(fop,(obstacles_named[path[i]][0],obstacles_named[path[i]][1]),(obstacles_named[path[i+1]][0],obstacles_named[path[i+1]][1]),(255,0,0),5)
    

        #########################################################################################
        else:
            # to output the raw feed
            fop = frame

        cv.imshow('Raw',frame)
        cv.imshow('Processed',fop)

        await client.sleep(0.001)
        
        # Close all windows and get out of the loop if ESC is pressed
        if cv.waitKey(1) == 27:
            cv.destroyAllWindows()
            break

Initialisation failed: division by zero
Initialisation failed: division by zero


C:\Users\zalex\AppData\Local\Temp\ipykernel_17264\174447522.py:64: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  Controlling_thymio.controlling_wheels_speed(int(vl),int(vr),aw,node)


ZeroDivisionError: division by zero

In [9]:
from tdmclient import ClientAsync, aw
client = ClientAsync()
import Controlling_thymio

node = await client.wait_for_node()
await node.lock()



aw(node.lock())



Node 0e83e7f0-32cd-4c48-b1de-4ae4fe01ea77

In [10]:
Controlling_thymio.controlling_wheels_speed(50,50,aw,node)


In [3]:
Controlling_thymio.controlling_wheels_speed(75,0,aw,node)


In [11]:
Controlling_thymio.controlling_wheels_speed(0,0,aw,node)


In [11]:
await node.wait_for_variables({"prox.horizontal"})
for i in range(10):
    print(list(node.v.prox.horizontal))
    await client.sleep(0.2)

[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 1496, 2575, 1925, 0, 0]
[0, 1929, 0, 2725, 3456, 0, 0]
[0, 4870, 2032, 1785, 2802, 0, 0]


In [20]:
await node.wait_for_variables({"acc"})
for i in range(10):
    print(Controlling_thymio.kidnapping(node))
    await client.sleep(0.001)

0
False
19
True
19
True
19
True
19
True
19
True
19
True
19
True
19
True
19
True


In [27]:
aw(node.unlock())